In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai import Crew, Agent, Task

international_chef = Agent(
    role="International Chef",
    goal="Create ethnic cuisine recipies that are easy to cook at home",
    backstory="""
    You are an famous chef that specializes in couisine from countries all around the world.
    You know how to cook the most traditional dishes from all cultures but you also know how to adapt them for people to be able to cook them at home.
    """,
    verbose=True,
    allow_delegation=False,
)

healthy_chef = Agent(
    role="Healthy Chef",
    goal="Turn any recipe into a healthy vegetarian recipe that is easy to cook with home ingredients.",
    backstory="""
    You are a chef specialized in healthy cooking.
    You can take any recipe and change the ingredients to make it vegetarian friendly without loosing the escense of the dish and what makes it delicious.
    """,
    verbose=True,
    allow_delegation=False,
)

In [3]:
normal_recipe = Task(
    description="Come up with a {dish} that serves {people} people.",
    agent=international_chef,
    expected_output="Your answer MUST have three sections, the ingredients required with their quantities, the preparation instructions and serving suggestions",
    output_file="normal_recipe.md",
)

healthy_recipe = Task(
    description="Replace the ingredients of a recipe to make it vegetarian without making it less delicious, adjust if needed.",
    agent=healthy_chef,
    expected_output="Your answer MUST have four sections, the ingredients required with their quantities, the preparation instructions, serving suggestions and an explanation of the replaced ingredients.",
    output_file="healthy_recipe.md",
)

In [4]:
crew = Crew(
    tasks=[
        normal_recipe,
        healthy_recipe,
    ],
    agents=[
        international_chef,
        healthy_chef,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        dish="Greek dinner",
        people="5",
    )
)

 [DEBUG]: == Working Agent: International Chef
 [INFO]: == Starting Task: Come up with a Greek dinner that serves 5 people.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: 

**Greek Dinner for 5 People**

**Ingredients:**

*For the Main Dish: Chicken Souvlaki*  
- 1.5 pounds (700g) boneless, skinless chicken thighs, cut into 1-inch pieces  
- 3 tablespoons olive oil  
- 3 tablespoons fresh lemon juice  
- 4 cloves garlic, minced  
- 2 teaspoons dried oregano  
- 1 teaspoon salt  
- ½ teaspoon black pepper  
- Wooden skewers (soaked in water for 30 minutes)

*For the Tzatziki Sauce:*  
- 1 cup Greek yogurt  
- 1 medium cucumber, grated and excess moisture squeezed out  
- 2 cloves garlic, minced  
- 1 tablespoon olive oil  
- 1 tablespoon fresh dill, chopped (or 1 teaspoon dried dill)  
- 1 tablespoon fresh lemon juice  
- Salt and pepper to taste  

*For the Greek Salad:*  
- 4 cups romaine lettuce, chopped  
- 1 cup cherry tomatoes, halved  
-